# 評価方法

分類されていないデータを認識し、どれだけ正しくカテゴリごとに分類できるかを算出した「平均精度」の高さを競い合います。

今回、活用するデータはLSWMD_25519となります。
LSWMD_25519のFailureType項目が分類されていない状態のデータに対し、正しいFailureTypeカテゴリを分類するプログラムを作成し、その平均精度を算出します。
平均精度とは、カテゴリごとに正しく分類できる精度を平均した値です。カテゴリごとに算出した精度（Aが正しく分類された数/Aのデータ数）を足し、カテゴリ数で割ります。

公平な評価を実施するために、以下の制限を設けています。
1. 外部パッケージをインストールするためのセルとsolution関数の中身のみを編集すること
2. 校舎のiMac上で最後のセルの実行時間が15分未満であること　（%%timeitの出力結果を確認してください）

※気になる点がある場合、Discordで気軽にお問合せください。

In [26]:
import numpy as np # https://numpy.org/ja/
import pandas as pd # https://pandas.pydata.org/
from sklearn.model_selection import train_test_split

外部パッケージを使用する場合、以下の方法でインストールを実施してください。

In [27]:
# 必要な外部パッケージは、以下の内容を編集しインストールしてください
# !pip install keras
# !pip install tensorflow[and-cuda]
# !pip install opencv-python

以下のsolution関数のみ編集してください。

In [28]:
def solution(x_test_df, train_df):
    import numpy as np
    from skimage.transform import resize
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import StandardScaler

    #画像を共通のサイズにリサイズ
    target_size = (64, 64)
    resized_images = [resize(img, target_size, anti_aliasing=True) for img in train_df['waferMap']]

    #NumPy配列に変換
    X_train = np.array(resized_images)
    y_train = np.array(train_df['failureType'])

    #PCA次元削減
    pca = PCA(n_components=50)
    X_train_pca = pca.fit_transform(X_train.reshape(X_train.shape[0], -1))
    
    # kNN作成と訓練
    knn_model = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=20, metric='minkowski', p=2)
    knn_model.fit(X_train_pca, y_train)

    #テストデータで予測
    resized_images_test = [resize(img, target_size, anti_aliasing=True) for img in x_test_df['waferMap']]

    X_test = np.array(resized_images_test)
    X_test_pca = pca.transform(X_test.reshape(X_test.shape[0], -1))
    
    y_pred = knn_model.predict(X_test_pca)

    return pd.DataFrame({'failureType': y_pred}, index=x_test_df.index)

solution関数は以下のように活用され、平均精度を計算します。

In [29]:
%%timeit -r 1 -n 1

# データのインポート
df=pd.read_pickle("../input/LSWMD_25519.pkl")

# テスト用と学習用のデータを作成（テストする際は、random_stateの値などを編集してみてください）
train_df, test_df = train_test_split(df, stratify=df['failureType'], test_size=0.10, random_state=42)

y_test_df = test_df[['failureType']]
x_test_df = test_df.drop(columns=['failureType'])

# solution関数を実行
user_result_df = solution(x_test_df, train_df)

average_accuracy = 0
# ユーザーの提出物のフォーマット確認
if type(y_test_df) == type(user_result_df) and y_test_df.shape == user_result_df.shape:
    # 平均精度の計算
    accuracies = {}
    for failure_type in df['failureType'].unique():
        y_test_df_by_failure_type = y_test_df[y_test_df['failureType'] == failure_type]
        user_result_df_by_failure_type = user_result_df[y_test_df['failureType'] == failure_type]
        matching_rows = (y_test_df_by_failure_type == user_result_df_by_failure_type).all(axis=1).sum()
        accuracies[failure_type] = (matching_rows/(len(y_test_df_by_failure_type)))
    
    average_accuracy = sum(accuracies.values())/len(accuracies)
    average_accuracy_sum += average_accuracy

print(f"平均精度：{average_accuracy*100:.2f}%")

平均精度：81.95%
平均精度：82.35%
平均精度：82.57%
平均精度：82.57%
平均精度：82.99%
平均精度：82.56%
平均精度：83.01%
平均精度：82.19%
平均精度：82.55%
平均精度：82.72%
平均精度期待値：82.55%
1min 42s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


平均精度期待値：82.37%
1min 43s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

平均精度期待値：82.48%
1min 42s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)